___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___

# Topic Modeling Assessment Project

Welcome to your Topic Modeling Assessment! For this project you will be working with a dataset of over 400,000 quora questions that have no labeled cateogry, and attempting to find 20 cateogries to assign these questions to. The .csv file of these text questions can be found underneath the Topic-Modeling folder.

Remember you can always check the solutions notebook and video lecture for any questions.

#### Task: Import pandas and read in the quora_questions.csv file.

In [4]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.util import ngrams
import nltk
nltk.download('wordnet')
import spacy
spacy.load('en_core_web_lg')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\blake.coston\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
df = pd.read_csv('quora_questions.csv')

df.head()

,Question
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."


# Preprocessing

#### Task: Use TF-IDF Vectorization to create a vectorized document term matrix. You may want to explore the max_df and min_df parameters.

In [6]:
# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Get the default English stopwords from spacy
nlp = spacy.load('en_core_web_lg')
stop_words = nlp.Defaults.stop_words
    
# Function to preprocess, tokenize, and lemmatize text
def preprocess(text):
    tokens = text.split()
    return ' '.join([lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words])

# Apply the preprocessing to the questions
df['preprocessed_question'] = df['Question'].apply(lambda text: preprocess(text))

In [7]:
# Vectorize the text data
# max_df: if a word is contianed in 95% of documents it is discarded
# min_df: if a word is only contained in 2 or less documents it is discarded

vectorizer = TfidfVectorizer(max_df = 0.95, min_df = 2, stop_words='english')
X = vectorizer.fit_transform(df['preprocessed_question'].fillna(''))

In [14]:
df.head()

,Question,preprocessed_question
0,What is the step by step guide to invest in sh...,step step guide invest share market india?
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,story kohinoor (koh-i-noor) diamond?
2,How can I increase the speed of my internet co...,increase speed internet connection vpn?
3,Why am I mentally very lonely? How can I solve...,mentally lonely? solve it?
4,"Which one dissolve in water quikly sugar, salt...","dissolve water quikly sugar, salt, methane car..."


# Non-negative Matrix Factorization

#### TASK: Using Scikit-Learn create an instance of NMF with 20 expected components. (Use random_state=42)..

In [8]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=20, random_state=42)

nmf.fit(X)

NMF(n_components=20, random_state=42)

#### TASK: Print our the top 15 most common words for each of the 20 topics.

In [20]:
words = vectorizer.get_feature_names_out()
for i, component in enumerate(nmf.components_):
    ordered = sorted(component, reverse=True)
    print(f'Topic {i + 1}')
    top_words = []
    for word_index, num in enumerate(component):
        if num in ordered[:15]:
            top_words.append(words[word_index])
    print(top_words)

Topic 1
['2016', 'best', 'book', 'buy', 'company', 'friend', 'laptop', 'movie', 'movies', 'phone', 'place', 'read', 'site', 'visit', 'website']
Topic 2
['add', 'answer', 'answered', 'ask', 'asked', 'delete', 'easily', 'google', 'improvement', 'marked', 'needing', 'post', 'question', 'quora', 'search']
Topic 3
['black', 'earn', 'easily', 'easy', 'facebook', 'free', 'home', 'internet', 'investment', 'job', 'making', 'money', 'online', 'website', 'youtube']
Topic 4
['balance', 'change', 'changed', 'death', 'earth', 'important', 'life', 'live', 'meaning', 'moment', 'purpose', 'real', 'thing', 'want', 'work']
Topic 5
['2016', 'america', 'better', 'clinton', 'donald', 'election', 'happen', 'hillary', 'presidency', 'president', 'presidential', 'think', 'trump', 'vote', 'win']
Topic 6
['culture', 'don', 'feel', 'friend', 'girl', 'guy', 'like', 'live', 'look', 'men', 'sex', 'tell', 'website', 'woman', 'work']
Topic 7
['beginner', 'book', 'coding', 'computer', 'english', 'hacking', 'java', 'lang

event


#### TASK: Add a new column to the original quora dataframe that labels each question into one of the 20 topic categories.

,Question
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."


,Question,Topic
0,What is the step by step guide to invest in sh...,5
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,16
2,How can I increase the speed of my internet co...,17
3,Why am I mentally very lonely? How can I solve...,11
4,"Which one dissolve in water quikly sugar, salt...",14
5,Astrology: I am a Capricorn Sun Cap moon and c...,1
6,Should I buy tiago?,0
7,How can I be a good geologist?,10
8,When do you use シ instead of し?,19
9,Motorola (company): Can I hack my Charter Moto...,17


# Alternative Methods of Grouping
K-means and LDA

In [77]:
# K-means
from sklearn.cluster import KMeans

def cluster_and_filter_relevance(df, n_clusters=5, n_key_words=10):
    """
    Perform K-means clustering on the articles' abstracts and filter the most relevant clusters with lemmatization.
    USes TF-IDF to vectorize the documents and K-means to cluster them into groups
    
    :param df: DataFrame containing the articles data.
    :param n_clusters: Number of clusters to create.
    :param n_key_words: Number of top keywords to use for filtering relevant clusters.
    
    :return: Filtered DataFrame with relevant clusters, and a dictionary containing cluster keywords.
    """
    # Vectorize the text data
    # max_df: if a word is contianed in 95% of documents it is discarded
    # min_df: if a word is only contained in 2 or less documents it is discarded
    vectorizer = TfidfVectorizer(max_df = 0.95, min_df = 2, stop_words='english')
    X = vectorizer.fit_transform(df['Preprocessed_Question'].fillna(''))

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
    df['K_Means_Cluster'] = kmeans.labels_
    
    # Analyze the clusters to determine relevance
    # Initialize a dictionary to store the keywords for each cluster
    cluster_keywords = {}
    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names_out()
    
    # Iterate through each cluster and store the top n keywords
    for i in range(n_clusters):
        cluster_keywords[i] = [terms[ind] for ind in order_centroids[i, :n_key_words]]
    
    # Here you could filter clusters based on relevance, or simply drop the combined column
    df_filtered = df.copy()  # If you want to perform further filtering, modify df_filtered

    # Return both the filtered DataFrame and the cluster_keywords dictionary
    return df_filtered, cluster_keywords

df_k_means, clusters_k_means = cluster_and_filter_relevance(df, 20, 10)

for group in sorted(clusters_k_means.keys()):
    print(f'cluster_k_means {group} keywords: {clusters_k_means[group]}')

print("\nCluster counts:")
print(df_k_means['Cluster'].value_counts().sort_index())

cluster_k_means 0 keywords: ['mean', 'say', 'dream', 'girl', 'symbol', 'guy', 'word', 'phrase', 'love', 'person']
cluster_k_means 1 keywords: ['long', 'school', 'distance', 'relationship', 'high', 'work', 'term', 'best', 'stay', 'time']
cluster_k_means 2 keywords: ['like', 'people', 'feel', 'think', 'look', 'girl', 'work', 'culture', 'sex', 'hate']
cluster_k_means 3 keywords: ['attend', 'like', 'school', 'university', 'college', 'conference', 'international', 'student', 'yale', 'best']
cluster_k_means 4 keywords: ['buy', 'best', 'laptop', 'phone', '15000', 'car', 'india', 'online', '15k', 'inr']
cluster_k_means 5 keywords: ['account', 'password', 'gmail', 'email', 'facebook', 'instagram', 'recover', 'delete', 'hack', 'forgot']
cluster_k_means 6 keywords: ['computer', 'science', 'engineering', 'data', 'best', 'university', 'difference', 'student', 'good', 'learning']
cluster_k_means 7 keywords: ['whatsapp', 'hack', 'message', 'account', 'phone', 'group', 'profile', 'chat', 'status', 'kn

In [ ]:
df.head()

In [ ]:
# LDA (takes a long time)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE


# Use CountVectorizer to convert the text data into a matrix of token counts
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['Preprocessed_Question'])

# Define the LDA model with the number of topics you want to extract
num_topics = 20  # You can adjust this number
lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda.fit(X)

# Get the topic distribution for each document
doc_topic_dist = lda.transform(X)

# Reduce the dimensionality of the topics for visualization using t-SNE
tsne_model = TSNE(n_components=2, random_state=42)
tsne_lda = tsne_model.fit_transform(doc_topic_dist)

# Plot the topics in a 2D space
plt.figure(figsize=(12, 8))
plt.scatter(tsne_lda[:, 0], tsne_lda[:, 1], c=doc_topic_dist.argmax(axis=1), cmap='viridis')
plt.colorbar(label='Topic')
plt.title('2D Visualization of LDA Topics')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

# Display the top words in each topic
num_top_words = 10  # Number of top words to display for each topic
feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-num_top_words - 1:-1]]
    print(f"Topic #{topic_idx + 1}: {', '.join(top_words)}")


# Great job!